In [1]:
import numpy as np
import tensorflow as tf
import pickle
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Demo

In [2]:
GRU_MODEL_PATH = 'GRU_W2V_best.keras'
TOKENIZER_PATH = 'tokenizer.pickle'
MAX_SEQUENCE_LENGTH = 150

In [3]:
try:
    gru_model = tf.keras.models.load_model(GRU_MODEL_PATH)
    print("Deep Learning model loaded successfully.")
    gru_model.summary() # In ra cấu trúc model để kiểm tra
except Exception as e:
    print(f"Error loading Deep Learning model: {e}")
    print("Please ensure the model file exists and is compatible.")

Deep Learning model loaded successfully.


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 150, 100)       │       287,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 150, 128)       │        63,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 64)             │        31,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 578,484 (2.21 MB)

 Trainable params: 96,994 (378.88 KB)

 Non-trainable params: 287,500 (1.10 MB)

 Optimizer params: 193,990 (757.78 KB)

In [4]:
# Load the tokenizer from TOKENIZER_PATH
with open(TOKENIZER_PATH, 'rb') as handle:
    tokenizer = pickle.load(handle)

In [5]:
import sys
sys.path.insert(0, "../../")

In [6]:
from src.shared.text_preprocessing import VietnamesePreprocessingWithoutSpellCheck

Current file path: D:\Documents\Ki_8\NLP\Comment-Sentiment-Analysis
Thu mục gốc: D:\Documents\Ki_8\NLP\Comment-Sentiment-Analysis
Using model path: D:\Documents\Ki_8\NLP\Comment-Sentiment-Analysis\attribute_extractor\vncorenlp


In [7]:
without_spellcheck_preprocessor = VietnamesePreprocessingWithoutSpellCheck()

In [8]:
def predict_sentiment(text, model, tokenizer, max_length):
    # clean_text = text # Tạm thời bỏ qua làm sạch phức tạp
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')
    prediction = model.predict(padded_sequence, verbose=0) # Thêm verbose=0
    labels = (prediction > 0.5).astype(int)[0]
    print(f"Predicted labels (positive, negative): {labels}")
    print(f"Probabilities:")
    print(f"P(positive=1): {prediction[0][0]}")
    print(f"P(negative=1): {prediction[0][1]}")
    predicted_dict = {"positive": int(labels[0]), "negative": int(labels[1])}
    return predicted_dict, prediction[0]

In [ ]:
# Nhập input liên tục từ người dùng cho đến khi người dùng nhập "exit"
while True:
    user_input = input("Nhập một comment hoặc 'exit' để thoát: ")
    
    if user_input.lower() == 'exit':
        print("Thoát chương trình.")
        break
    
    print("==================================")
    print("Comment gốc:", user_input)
    proceeded_input = without_spellcheck_preprocessor(user_input)
    print("Comment:", proceeded_input)
    pred_labels, pred_probs = predict_sentiment(proceeded_input, gru_model, tokenizer, MAX_SEQUENCE_LENGTH)

Comment gốc: xin chào
Comment: chào
Predicted labels (positive, negative): [1 0]
Probabilities:
P(positive=1): 0.996173083782196
P(negative=1): 0.00766832334920764
Comment gốc: sản phẩm tốt
Comment: sản_phẩm tốt
Predicted labels (positive, negative): [1 0]
Probabilities:
P(positive=1): 0.997830331325531
P(negative=1): 0.000977963674813509
Comment gốc: pin nhanh nóng quá
Comment: pin nhanh nóng quá
Predicted labels (positive, negative): [0 1]
Probabilities:
P(positive=1): 0.02740609273314476
P(negative=1): 0.9339203834533691
Comment gốc: máy mượt, chiến game mạnh
Comment: máy mượt chiến game mạnh
Predicted labels (positive, negative): [1 0]
Probabilities:
P(positive=1): 0.9158605337142944
P(negative=1): 0.18729235231876373
Comment gốc: tháng này bên hãng có khuyến mãi gì không
Comment: tháng bên hãng khuyến_mãi gì không
Predicted labels (positive, negative): [0 0]
Probabilities:
P(positive=1): 0.0010322934249415994
P(negative=1): 0.04406847432255745
Comment gốc: nhân viên khó chịu sao ấ

In [10]:
# Nhập input liên tục từ người dùng cho đến khi người dùng nhập "exit"
while True:
    user_input = input("Nhập một comment hoặc 'exit' để thoát: ")
    
    if user_input.lower() == 'exit':
        print("Thoát chương trình.")
        break
    
    print("==================================")
    print("Comment gốc:", user_input)
    proceeded_input = without_spellcheck_preprocessor(user_input)
    print("Comment:", proceeded_input)
    pred_labels, pred_probs = predict_sentiment(user_input, gru_model, tokenizer, MAX_SEQUENCE_LENGTH)

Comment gốc: tự search mạng đi
Comment: search mạng đi
Predicted labels (positive, negative): [0 0]
Probabilities:
P(positive=1): 0.014776567928493023
P(negative=1): 0.05129661038517952
Comment gốc: bạn ngu quá
Comment: ngu quá
Predicted labels (positive, negative): [0 1]
Probabilities:
P(positive=1): 0.04069363325834274
P(negative=1): 0.7852215766906738
Comment gốc: ok
Comment: ổn
Predicted labels (positive, negative): [1 0]
Probabilities:
P(positive=1): 0.8647451400756836
P(negative=1): 0.15054906904697418
Comment gốc: fuck
Comment: fuck
Predicted labels (positive, negative): [1 0]
Probabilities:
P(positive=1): 0.8647451400756836
P(negative=1): 0.15054906904697418
Comment gốc: ok
Comment: ổn
Predicted labels (positive, negative): [1 0]
Probabilities:
P(positive=1): 0.8647451400756836
P(negative=1): 0.15054906904697418
Comment gốc: bạn chán ghê
Comment: chán ghê
Predicted labels (positive, negative): [1 0]
Probabilities:
P(positive=1): 0.8255239129066467
P(negative=1): 0.0688493326306